# 4장 단어 수준 임베딩
## 4.1 NPLM

Neural Probabilistic Language Model(NPLM)은 자연어 처리 분야에 임베딩 개념을 널리 퍼뜨리는 데 일조한 선구자적 모델로, 딥러닝의 아버지 요슈아 벤지오 연구 팀이 제안한 기법(Bengio et al., 2003)이다.


### 모델 기본 구조
NPLM은 '단어가 어떤 순서로 쓰였는가'에서 설명한 통계 기반의 전통적인 언어 모델의 한계를 극복하는 과정에서 탄생했다. Bengio et al.(2003)은 기존 언어 모델의 단점을 다음과 같이 설명했다.

- 학습 데이터에 존재하지 않는 n-gram이 포함된 문장이 나타날 확률 값을 0으로 부여한다. 물론 **백오프**나 **스무딩**으로 이런 문제를 일부 보완할 수 있지만 완전한 것은 아니다.
- 문장의 **장기 의존성(long-term dependency)**를 포착해내기 어렵다. 다시 말해 n-gram 모델의 n을 5 이상으로 길게 설정할 수 없다. n이 커질수록 그 등장 확률이 0인 단어 시퀀스가 기하급수적으로 늘어난다. 
- 단어/문장 간 유사도를 계산할 수 없다. 

### NPLM의 학습
학습 파라미터의 종류가 많고 그 크기도 큰 편이다. 

## 4.2 Word2Vec
2013년 구글 연구 팀이 발표한 기법으로 가장 널리 쓰이고 있는 단어 임베딩 모델이다. 두 개의 논문으로 나누어 발표했는데, 하나는 Skip-Gram과 CBOW라는 모델이 제안됐고 다른 하나는 이 두 모델을 근간으로 하되 네거티브 샘플링 등 학습 최적화 기법을 제안한 내용이 핵심 골자이다. 

### 모델 기본 구조
- CBOW는 주변에 있는 **문맥 단어**들을 가지고 **타깃 단어** 하나를 맞추는 과정에서 학습된다. 
- Skip-gram 모델은 타깃 단어를 가지고 주변 문맥 단어가 무엇일지 예측하는 과정에서 학습된다. 
- CBOW의 경우 입력, 출력 학습 데이터 쌍이 {문맥 단어 4개, 타깃 단어} 하나인 반면, Skip-gram의 학습 데이터는 {타깃 단어, 타깃 직전 두 번째 단어}, {타깃 단어, 타깃 직전 단어}, {타깃 단어, 타깃 다음 단어}, {타깃 단어, 타깃 다음 두 번째 단어} 이렇게 4개쌍이 된다. Skip-gram이 같은 말뭉치로도 더 많은 학습 데이터를 확보할 수 있어서 임베딩 품질이 CBOW보다 좋은 경향이 있다. 따라서 Skip-gram 모델을 중심으로 Word2Vec 기법을 설명한다. 

In [1]:
# 튜토리얼

#git pull origin master
#bash preprocess.sh dump-tokenized

#cd /notebooks/embedding
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt 

In [ ]:
# Word2Vec Skip-gram 모델 학습

corpus_fname = '/notebooks/embedding/data/tokenized/corpus_mecab.txt'
model_fname = '/notebooks/embedding/data/word-embbedings/word2vec/word2vec'

from genism.models import Word2Vec

corpus = [sent.strip().split(" ") for sent in open(corpus_fname, 'r').readlines()]
model = Word2Vec(corpus, size=100, workers=4, sg=1)
model.save(model_fname)

In [ ]:
# Word2Vec Skip-gram 모델 학습 스크립트
#cd /notebooks/embedding
#python models/word_utils.py --method train_word2vec \
--input_path data/tokenized/corpus_mecab.txt \
--input_path data/word-embeddings/word2vec/word2vec

In [ ]:
# 코사인 유사도 상위 단어 목록 체크 코드

from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator("/notebooks/embedding/data/word-embbedings/word2vec/word2vec", method="word2vec", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

#### 코사인 유사도 유의점
코사인 유사도 값이 높다도 해서 두 단어가 유의 관계인 것은 아니다. 예를 들어 '덥다'와 '춥다'는 반의 관계이지만, '기온'이라는 속성을 매개로 강한 관련을 맺고 있기 때문에 코사인 유사도 값이 높다. **관련성**이 높은 단어를 출력한다는 의미로 이해하는 것이 좋다. 

## 4.3 FastText

- FastText(Bojanowski et al, 2017)는 페이스북에서 개발해 공개한 단어 임베딩 기법이다.
- 각 단어를 **문자 단위 n-gram**으로 표현한다. 이 밖의 내용은 Word2Vec과 같다.

### 4.3.1 모델 기본 구조
- 예를 들어 ``시나브로``라는 단어의 문자 단위 n-gram(n=3)은 다음과 같은 **5개의 문자 단위 n-gram 벡터**로, 단어의 임베딩은 이 다섯 개의 벡터의 합으로 표현한다. 
    - <시나, 시나브, 나브로, 브로>, <시나브로>
- **네거티브 샘플링 기법**: Word2Vec 방식 + 타깃 단어(t), 문맥 단어(c) 쌍을 학습할 때 타깃 단어(t)에 속한 문자 단위 n-gram 벡터(z)들을 모두 업데이트한다. 
- FastText 모델에서 **포지티브 샘플이 주어졌을 때** 포지티브 샘플일 확률(분수 형태의 공식)을 최대화하려면 분모를 최소화해야 한다. 분모를 최소화하기 위해서는 z들과 $v_{c}$ 간 내적 값을 높여야 한다. 벡터의 내적은 코사인 유사도와 비례한다. 따라서 내적 값의 상향은 타깃 단어(t)에 속하는 문자 단위 n-gram 벡터와 문맥 단어의 포지티브 샘플(c)에 해당하는 단어 벡터 간 유사도를 높여야 한다는 의미로 이해할 수 있다.
    - 예를 들어 ``시나브로``가 타깃 단어(t), ``쌓였다``가 문맥 단어의 포지티브 샘플(c)이라면 ``<시나``, ``시나브``,``나브로``, ``브로>``, ``<시나브로>`` 등 문자 n-gram 벡터(z)들 각각을 ``쌓였다``에 해당하는 단어 벡터($v_{c}$)와의 **유사도를 높인다**. 
- 또한 **네거티브 샘플이 주어졌을 때** 네거티브 샘플일 확률(분수 형태의 공식)을 최대화하려면 분자를 최대화해야 한다. 즉, 네거티브 샘플을 모델에 입력하면 모델은 이 데이터가 정말 네거티브 샘플이라고 잘 맞춰야 한다. 그러기 위해서는 분자를 최대화해야 하는데 z들과 $v_{c}$ 간 내적 값을 낮춰야 한다. 마찬가지로 벡터의 내적은 코사인 유사도와 비례한다. 따라서 내적 값의 하향은 타깃 단어(t)에 속하는 문자 단위 n-gram 벡터와 문맥 단어의 네거티브 샘플(c)에 해당하는 단어 벡터 간 유사도를 낮춰야한다는 의미로 이해할 수 있다. 
    - 예를 들어 ``시나브로``가 타깃 단어(t), ``컴퓨터``가 문맥 단어의 네거티브 샘플(c)이라면, ``<시나``, ``시나브``,``나브로``, ``브로>``, ``<시나브로>`` 등 문자 n-gram 벡터(z)들 각각을 ``컴퓨터``에 해당하는 단어 벡터($v_{c}$)와의 **유사도를 낮춘다**.
- **로그우도 함수**: FastText 모델이 최대화해야 할 로그우도 함수, 모델을 한 번 업데이트할 때 **1개의 포지티브 샘플($t_{p}, c_{p}$)과 k개의 네거티브 샘플($t_{n_{i}}, c_{n_{i}}$)을 학습한다**는 의미 

### 4.3.2 튜토리얼

In [2]:
# 데이터 다운로드 
#git pull origin master
#bash preprocess.sh dump-tokenized

In [3]:
# 데이터 합치기
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt

In [5]:
# FastText Skip-gram 모델 학습
#mkdir -p data/word-embeddings/fasttexrt
#models/fastText/fasttext skipgram - input data/tokenized/corpus_mecab.txt -output data/word-embeddings/fasttext/fasttext

In [ ]:
# FastText Skip-gram 모델의 코사인 유사도 상위 단어 목록 체크
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator(
    vecs_text_fname = 'data/word-embeddings/fasttext/fasttext.vec'
    vecs_bin_fname = 'data/word-embeddings/fasttext/fasttext.bin',
    method='fasttext', dim=100, tokenizer_name='mecab'
)
model.most_similar('희망', topn=5)
model.most_similar('하였다', topn=5)

- FastText는 오타나 **미등록 단어**에도 강건하다(robust). 각 단어의 임베딩을 문자 단위 n-gram 벡터의 합으로 표현하기 때문이다. 
    - 예를 들어 미등록 단어 ``서울 특별시``에 대해서도 FastText 임베딩을 추정할 수 있다. ``서울특별시``는 ``서울`` 같은 문자 단위 n-gram을 포함하고 있다. ``서울``이 어휘 집합에 있다면 나머지 n-gram(울특, 특별 등)이 모두 미등록 단어라 할지라도 ``서울특별시``에 대한 임베딩을 추정할 수 있다. 

In [ ]:
model._is_in_vocabulary('서울특별시')
model.get_word_vector('서울특별시')
model.most_similar('서울특별시')

### 4.3.3 한글 자소와 FastText
-여기서부터는 내일 하기! 